# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Joharle  ->  S. Joharle  |  ['S. Joharle']
J. Wolf  ->  D. J. Wolf  |  ['J. Wolf']
J. Li  ->  J. Li  |  ['J. Li']
T. Henning  ->  T. Henning  |  ['T. Henning']


Arxiv has 71 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.04778
extracting tarball to tmp_2511.04778...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.05029
extracting tarball to tmp_2511.05029... done.


J. Wolf  ->  D. J. Wolf  |  ['J. Wolf']


Retrieving document from  https://arxiv.org/e-print/2511.05144
extracting tarball to tmp_2511.05144... done.
Retrieving document from  https://arxiv.org/e-print/2511.05154


extracting tarball to tmp_2511.05154... done.


T. Henning  ->  T. Henning  |  ['T. Henning']


/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\hsize]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[angle=-90,width=3cm]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_d

Issues with the citations
repeated bibliography entry: picos_eso_2024


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.05029-b31b1b.svg)](https://arxiv.org/abs/2511.05029) | **Discovery of an X-ray Luminous Radio-Loud Quasar at $z=3.4$: A Possible Transitional Super-Eddington Phase**  |
|| S. Obuchi, et al. -- incl., <mark>J. Wolf</mark> |
|*Appeared on*| *2025-11-10*|
|*Comments*| *20 pages, 11 figures, accepted for publication in ApJ*|
|**Abstract**|            We report the multiwavelength properties of eFEDS J084222.9+001000 (hereafter ID830), a quasar at $z=3.4351$, identified as the most X-ray luminous radio-loud quasar in the eROSITA Final Equatorial Depth Survey (eFEDS) field. ID830 shows a rest-frame 0.5-2 keV luminosity of $\log (L_\mathrm{0.5-2\,keV}/\mathrm{erg}~\mathrm{s}^{-1}) = 46.20 \pm 0.12$, with a steep X-ray photon index ($\Gamma =2.43 \pm 0.21$), and a significant radio counterpart detected with VLA/FIRST 1.4 GHz and VLASS 3 GHz bands. The rest-frame UV to optical spectra from SDSS and Subaru/MOIRCS $J$-band show a dust reddened quasar feature with $A_\mathrm{V} = 0.39 \pm 0.08$ mag and the expected bolometric AGN luminosity from the dust-extinction-corrected UV luminosity reaches $L_\mathrm{bol,3000}= (7.62 \pm 0.31) \times 10^{46}$ erg s$^{-1}$. We estimate the black hole mass of $M_\mathrm{BH} = (4.40 \pm 0.72) \times 10^{8} M_{\odot}$ based on the MgII$\lambda$2800 emission line width, and an Eddington ratio from the dust-extinction-corrected UV continuum luminosity reaches $\lambda_\mathrm{Edd,UV}=1.44 \pm 0.24$ and $\lambda_{\mathrm{Edd,X}} = 12.8 \pm 3.9$ from the X-ray luminosity, both indicating the super-Eddington accretion. ID830 shows a high ratio of UV-to-X-ray luminosities, $\alpha_\mathrm{OX}=-1.20 \pm 0.07$ (or $\alpha_\mathrm{OX}=-1.42 \pm 0.07$ after correcting for jet-linked X-ray excess), higher than quasars and little red dots in super-Eddington phase with similar UV luminosities, with $\alpha_\mathrm{OX}<-1.8$. Such a high $\alpha_\mathrm{OX}$ suggests the coexistence of a prominent radio jet and X-ray corona, in this high Eddington accretion phase. We propose that ID830 may be in a transitional phase after an accretion burst, evolving from a super-Eddington to a sub-Eddington state, which could naturally describe the high $\alpha_\mathrm{OX}$.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.05154-b31b1b.svg)](https://arxiv.org/abs/2511.05154) | **Upper limits on atmospheric abundances of KELT-11b and WASP-69b from a retrieval approach**  |
|| F. Lesjak, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-11-10*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics, 18 pages, 17 figures*|
|**Abstract**|            WASP-69b and KELT-11b are two low-density hot Jupiters, which are expected to show strong atmospheric features in their transmission spectra. Such features offer valuable insights into the chemical composition, thermal structure, and cloud properties of exoplanet atmospheres. High-resolution spectroscopic observations can be used to study the line-forming regions in exoplanet atmospheres and potentially detect signals despite the presence of clouds. We aimed to detect various molecular species and constrain the chemical abundances and cloud deck pressures using high-resolution spectroscopy. We observed multiple transits of these planets with CARMENES and applied the cross-correlation method to detect atmospheric signatures. Further, we used an injection-recovery approach and retrievals to place constraints on the atmospheric properties. We detected a tentative H$_2$O signal for KELT-11b but not for WASP-69b, and searches for other molecules such as H$_2$S and CH$_4$ resulted in non-detections for both planets. By investigating the signal strength of injected synthetic models, we constrained which atmospheric abundances and cloud deck pressures are consistent with our cross-correlation results. In addition, we show that a retrieval-based approach leads to similar constraints of these parameters.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.04778-b31b1b.svg)](https://arxiv.org/abs/2511.04778) | **Kinematic and extinction analysis of a potential spiral arm beyond the Galactic bar**  |
|| <mark>S. Joharle</mark>, F. Nogueras-Lara, K. Fiteni |
|*Appeared on*| *2025-11-10*|
|*Comments*| *Accepted for publication in A&A. 11 pages, 9 figures. Abstract abridged to satisfy arXiv requirements*|
|**Abstract**|            Determining the structure of the Milky Way is essential for understanding its morphology, dynamics, and evolution. However, studying its innermost regions is challenging due to high extinction and crowding. The detection of a double red clump (RC; core-helium-burning stars) feature at very low Galactic latitudes suggests the presence of a spiral arm beyond the Galactic bar, providing new insights into the Galaxy's structure along this complex line of sight. We evaluate this possibility by analysing the proper motion and extinction distributions of the detected RC features. We constructed proper motion and extinction difference maps to investigate the kinematic and reddening properties of the RC populations, and the kinematic differences were validated using N-body simulations of a Milky Way-like galaxy. We find that the two RC features are kinematically distinct, with a relative proper motion difference of $-0.16\pm0.02\, mas/yr$ in the component parallel to the Galactic plane. This difference can be explained by Galactic rotation if the two RCs lie at different distances, consistent with the simulations. The extinction towards the secondary RC is also $\sim0.05$ mag higher than that of the primary RC. Additionally, we estimate that the extinction difference between the RC features corresponds to only $\sim5\%$ of the total extinction from Earth to the first RC, suggesting little interstellar material between the farthest edge of the Galactic bar and the kinematically distinct structure traced by the secondary RC. Using $JK_s$ photometry, we derive $A_J/A_{K_s}=3.34\pm0.07$, consistent with previous results and showing no significant variation across fields or along the line of sight. The results support the secondary clump tracing a distant structure, possibly a spiral arm, although we cannot exclude that the population belongs to the disc.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.05144-b31b1b.svg)](https://arxiv.org/abs/2511.05144) | **Searching for Electromagnetic Counterpart Candidates to GW231123**  |
|| L. He, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-11-10*|
|*Comments*| *10 pages, 3 figures, comments are welcome*|
|**Abstract**|            The detection of GW231123, a gravitational-wave (GW) event with exceptionally massive and rapidly spinning black holes, suggests the possible formation within an active galactic nucleus (AGN) disk, which provides a favorable environment for potentially generating an observable electromagnetic (EM) counterpart. We conduct a search for such a counterpart by crossmatching the GW localization with a comprehensive catalog of AGN flares from the Zwicky Transient Facility. Our analysis yields six plausible optical flare candidates that are spatially and temporally coincident with GW231123 and exhibit significant deviations from their AGN baseline flux. Although these candidates represent a crucial first step, their true nature remains inconclusive. Confirming any one of these flares via future observations would provide a landmark validation of the AGN formation channel and unlock the multi-messenger potential of this extraordinary merger.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.05029/./alphaOX_after.png', 'tmp_2511.05029/./multiwaveSED_id830.png', 'tmp_2511.05029/./LF_id830xrayopt_blazar_RQQver.png']
copying  tmp_2511.05029/./alphaOX_after.png to _build/html/
copying  tmp_2511.05029/./multiwaveSED_id830.png to _build/html/
copying  tmp_2511.05029/./LF_id830xrayopt_blazar_RQQver.png to _build/html/
exported in  _build/html/2511.05029.md
    + _build/html/tmp_2511.05029/./alphaOX_after.png
    + _build/html/tmp_2511.05029/./multiwaveSED_id830.png
    + _build/html/tmp_2511.05029/./LF_id830xrayopt_blazar_RQQver.png
found figures ['', '', '', '', '', '', '']
file not found 
file not found 
file not found 
file not found 
file not found 
file not found 
file not found 
exported in  _build/html/2511.05154.md
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\todo}{\ifmmode \text{\color{purple}\Huge{\(\bullet\)}} \else{\color{purple}{\Huge\bullet}}\fi}$
$\newcommand{\finish}{\ifmmode \text{\color{blue}\Huge{\(\bullet\)}} \else{\color{blue}{\Huge\bullet}}\fi}$
$\newcommand{\REFS}{\todo \rev{REFS}}$
$\newcommand{\specz}{z_\mathrm{spec}}$
$\newcommand{\photz}{z_\mathrm{phot}}$
$\newcommand{\Mstar}{M_{\star}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Mbh}{M_\mathrm{BH}}$
$\newcommand{\Mdotbh}{\dot{M}_\mathrm{BH}}$
$\newcommand{\lamEdd}{\lambda_\mathrm{Edd}}$
$\newcommand{\Lbol}{L_\mathrm{AGN,bol}}$
$\newcommand{\LEdd}{L_\mathrm{Edd}}$
$\newcommand{\Luv}{L_\mathrm{UV}}$
$\newcommand{\maguv}{M_\mathrm{1450}}$
$\newcommand{\LuvalphaOX}{L_\mathrm{2500 \text{Å}}}$
$\newcommand{\Lx}{L_\mathrm{X}}$
$\newcommand{\Lsoftx}{L_\mathrm{0.5-2 keV}}$
$\newcommand{\fsoftx}{f_\mathrm{0.5-2 keV}}$
$\newcommand{\Lhardx}{L_\mathrm{2-10 keV}}$
$\newcommand{\NH}{N_\mathrm{H}}$
$\newcommand{\NHunit}{N_\mathrm{H}/\mathrm{cm}^{-2}}$
$\newcommand{\alphaox}{\alpha_\mathrm{OX}}$
$\newcommand{\mstar}{M_{\star}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\mbh}{M_\mathrm{BH}}$
$\newcommand{\mdotbh}{\dot{M}_\mathrm{BH}}$
$\newcommand{\lbol}{L_\mathrm{AGN,bol}}$
$\newcommand{\ledd}{L_{\rm Edd}}$
$\newcommand{\eddington}{\lambda_\mathrm{Edd}}$
$\newcommand{\liragn}{L_\mathrm{AGN,IR}}$
$\newcommand{\liragnunit}{\liragn/\mathrm{erg}~\mathrm{s}^{-1}}$
$\newcommand{\ltwelve}{L_{12 \mu{\rm m}}}$
$\newcommand{\etar}{\eta_\mathrm{rad}}$
$\newcommand{\etarpone}{\eta_\mathrm{rad,0.1}}$
$\newcommand{\lfirst}{L_\mathrm{1.4GHz}}$
$\newcommand{\lfirstunit}{L_\mathrm{1.4GHz}/\mathrm{W}~\mathrm{Hz}^{-1} }$
$\newcommand{\lvfirst}{\nu L_\nu (\mathrm{1.4GHz})}$
$\newcommand{\lvfirstunit}{\nu L_\nu (\mathrm{1.4GHz})/\mathrm{erg}~\mathrm{s}^{-1} }$
$\newcommand{\ljet}{L_\mathrm{jet}}$
$\newcommand{\ljetunit}{L_\mathrm{jet}/\mathrm{erg}~\mathrm{s}^{-1}}$
$\newcommand{\etaj}{\eta_\mathrm{jet}}$
$\newcommand{\lambdaedd}{\lambda_\mathrm{Edd}}$
$\newcommand{\av}{A_\mathrm{V}}$
$\newcommand{\R}{\mathcal{R_{\rm rest}}}$
$\newcommand{\Rint}{\mathcal{R_\mathrm{int}}}$
$\newcommand{\whz}{\mathrm{W}~\mathrm{Hz}^{-1}}$
$\newcommand{\sBHARunit}{\mathrm{erg}~\mathrm{s}^{-1}~M_\odot^{-1}}$
$\newcommand{\SFR}{\mathrm{SFR}}$
$\newcommand{\sSFR}{\mathrm{sSFR}}$
$\newcommand{\qir}{q_\mathrm{IR}}$
$\newcommand{\fbstar}{f_\mathrm{b,\star}}$
$\newcommand{\rt}[1]{#1}$
$\newcommand{\del}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$</div>



<div id="title">

# Discovery of an X-ray Luminous Radio-Loud Quasar at $z=3.4$: A Possible Transitional Super-Eddington Phase

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.05029-b31b1b.svg)](https://arxiv.org/abs/2511.05029)<mark>Appeared on: 2025-11-10</mark> -  _20 pages, 11 figures, accepted for publication in ApJ_

</div>
<div id="authors">

S. Obuchi, et al. -- incl., <mark>J. Wolf</mark>

</div>
<div id="abstract">

**Abstract:** We report the multiwavelength properties of eFEDS J084222.9+001000 (hereafter ID830),a quasar at $z=3.4351$ , identified as the most X-ray luminous radio-loud quasar in the eROSITAFinal Equatorial Depth Survey (eFEDS) field.ID830 shows a rest-frame 0.5--2 keV luminosity of $\log (\Lsoftx/\mathrm{erg} \mathrm{s}^{-1}) = 46.20 \pm 0.12$ ,with a steep X-ray photon index ( $\Gamma =2.43 \pm 0.21$ ), and a significant radio counterpart detected with VLA/FIRST 1.4 GHz and VLASS 3 GHz bands.The rest-frame UV to optical spectra from SDSS and Subaru/MOIRCS $J$ -band show a dust reddened quasar feature with $\av = 0.39 \pm 0.08$ mag and the expected bolometric AGN luminosity from the dust-extinction-corrected UV luminosity reaches $L_\mathrm{bol,3000 \text{Å}}= (7.62 \pm 0.31) \times 10^{46}$ erg s $^{-1}$ .We estimate the black hole mass of $\Mbh = (4.40 \pm 0.72) \times 10^{8} \msun$ based on the MgII $\lambda$ 2800 emission line width, and an Eddington ratio from the dust-extinction-corrected UV continuum luminosity reaches $\lambda_\mathrm{Edd,UV}=1.44 \pm 0.24$ and $\lambda_{\mathrm{Edd,X}} = 12.8 \pm 3.9$ from the X-ray luminosity, both indicating the super-Eddington accretion.ID830 shows a high ratio of UV-to-X-ray luminosities, $\alphaox=\rt{-1.20 \pm 0.07}$ (or $\alphaox=\rt{-1.42 \pm 0.07}$ after correcting for jet-linked X-ray excess),higher than quasars and little red dots in super-Eddington phase with similar UV luminosities, with $\alphaox<-1.8$ .Such a high $\alphaox$ suggests the coexistence of a prominent radio jet and X-ray corona, in this high Eddington accretion phase.We propose that ID830 may be in a transitional phase after an accretion burst, evolving from a super-Eddington to a sub-Eddington state, which could naturally describe the high $\alphaox$ .

</div>

<div id="div_fig1">

<img src="tmp_2511.05029/./alphaOX_after.png" alt="Fig6" width="100%"/>

**Figure 6. -** $\alphaox$ vs. UV monochromatic luminosity $\LuvalphaOX$ for ID830. The open star represents the $\rt${observed}$\alphaox$ of ID830. The filled star represents the $\rt${jet-subtracted}$\alphaox$, after removing the jet contamination. $\rt${Both $\alphaox$ points are higher than} the best-fit relation of [Lusso, et. al (2010)](https://ui.adsabs.harvard.edu/abs/2010A&A...512A..34L)(blue line) and [Nanni, et. al (2017)](https://ui.adsabs.harvard.edu/abs/2017A&A...603A.128N)(brown line). $\rt${The dotted lines are the $1 \sigma$ upper and lower bounds of the two best-fit relations.} The square plot is a eROSITA-detected quasar  ([Wolf, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...647A...5W)) , and black plots are PHL 1811 and its analogs  ([Leighly, et. al 2007](https://ui.adsabs.harvard.edu/abs/2007ApJ...663..103L), [Wu, et. al 2011](https://ui.adsabs.harvard.edu/abs/2011ApJ...736...28W), [Luo, et. al 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...805..122L)) . Other X-ray weak sources are also shown as purple inverted triangles  ([Zappacosta, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020A&A...635L...5Z)) , bold cross markers  ([Laurenti, et. al 2022](https://ui.adsabs.harvard.edu/abs/2022A&A...657A..57L)) , and plus markers  ([Nardini, et. al 2019](https://ui.adsabs.harvard.edu/abs/2019A&A...632A.109N)) .
Radio-loud quasars with X-ray excesses detected by _Chandra_, _XMM-Newton_, and ROSAT  ([Miller, et. al 2011](https://ui.adsabs.harvard.edu/abs/2011ApJ...726...20M))  are shown as green circles. The orange shaded region shows the eFEDS sensitive area.
 (*fig:alphaOX*)

</div>
<div id="div_fig2">

<img src="tmp_2511.05029/./multiwaveSED_id830.png" alt="Fig10" width="100%"/>

**Figure 10. -** 
The broad-band SED of ID830, along with the SED templates of radio-loud and radio-quiet quasars  ([Shang, et. al 2011](https://ui.adsabs.harvard.edu/abs/2011ApJS..196....2S))  and the blazar sequence  ([Ghisellini, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.469..255G)) . The blue, green, and orange shaded regions represent the individual bins of the blazar sequence. The light red plots are the $\rt${observed} X-ray spectrum of ID830. The dark red plots are the $\rt${jet-subtracted} X-ray spectrum, after removing the jet contamination. The upper limits are indicated by arrows.
 (*fig:multiwave_SED*)

</div>
<div id="div_fig3">

<img src="tmp_2511.05029/./LF_id830xrayopt_blazar_RQQver.png" alt="Fig11" width="100%"/>

**Figure 11. -** 
The left panel shows the rest-frame 2--10 keV X-ray luminosity function  ([Ueda, et. al 2014](https://ui.adsabs.harvard.edu/abs/2014ApJ...786..104U))  of AGNs. $\rt${The open red diamond represents} the number density plotted at the observed X-ray luminosity $\Lhardx$ of ID830. The filled red diamond represents the number density plotted at the $\rt${jet-subtracted} X-ray luminosity, after removing the jet contamination. The right panel shows the rest-frame 1450 Å magnitude UV luminosity function  ([Kulkarni, Worseck and Hennawi 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.488.1035K))  of quasars. The filled red diamond represents the number density at the UV magnitude $M_\mathrm{1450 \text{Å}}$ of ID830.
The solid line in each panel indicates the intrinsic luminosity function of the total AGN population, while the dashed line indicates that of the radio AGN population, assuming a radio AGN fraction (RL frac) of 10\%. The purple diamonds are the number densities assuming that ID830 is a blazar.
$\rt${The error bars span the cases for the range for Lorentz factors $\Gamma=5$--15 and do not include the error associated with the number density derived from the observed X-ray luminosity.} (*fig:luminosity_function*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.05029"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\sectionautorefname}{Section}$
$\newcommand{\subsectionautorefname}{Section}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# Upper limits on atmospheric abundances of KELT-11b and WASP-69b from a retrieval approach

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.05154-b31b1b.svg)](https://arxiv.org/abs/2511.05154)<mark>Appeared on: 2025-11-10</mark> -  _Accepted for publication in Astronomy & Astrophysics, 18 pages, 17 figures_

</div>
<div id="authors">

F. Lesjak, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** WASP-69 b and KELT-11 b are two low-density hot Jupiters, which are expected to show strong atmospheric features in their transmission spectra. Such features offer valuable insights into the chemical composition, thermal structure, and cloud properties of exoplanet atmospheres. High-resolution spectroscopic observations can be used to study the line-forming regions in exoplanet atmospheres and potentially detect signals despite the presence of clouds. We aimed to detect various molecular species and constrain the chemical abundances and cloud deck pressures using high-resolution spectroscopy. We observed multiple transits of these planets with CARMENES and applied the cross-correlation method to detect atmospheric signatures. Further, we used an injection-recovery approach and retrievals to place constraints on the atmospheric properties. We detected a tentative $H_2$ O signal for KELT-11 b but not for WASP-69 b, and searches for other molecules such as $H_2$ S and $CH_4$ resulted in non-detections for both planets. By investigating the signal strength of injected synthetic models, we constrained which atmospheric abundances and cloud deck pressures are consistent with our cross-correlation results. In addition, we show that a retrieval-based approach leads to similar constraints of these parameters.

</div>

<div id="div_fig1">

<img src="" alt="Fig5.1" width="25%"/><img src="" alt="Fig5.2" width="25%"/><img src="" alt="Fig5.3" width="25%"/><img src="" alt="Fig5.4" width="25%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div>
<div id="div_fig2">

<img src="" alt="Fig13.1" width="33%"/><img src="" alt="Fig13.2" width="33%"/><img src="" alt="Fig13.3" width="33%"/>

**Figure 13. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.05154"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

121  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

18  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
